![alt text](../utils/conve.png)

In [1]:
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import sklearn
import pprint
import itertools

import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

from data import WN18

In [2]:
class Config:
    n_epochs = 30
    batch_size = 128
    embed_dim = 200

In [3]:
"""
e: entity
s: subject
p: predicate
o: object
"""

def read_triples(path):
    triples = []
    with open(path, 'rt') as f:
        for line in f.readlines():
            s, p, o = line.split()
            triples += [(s.strip(), p.strip(), o.strip())]
    return triples


def load_triple():
    WN18.download()
    triples_tr = read_triples('../data/WN18/wn18/train.txt')
    triples_va = read_triples('../data/WN18/wn18/valid.txt')
    triples_te = read_triples('../data/WN18/wn18/test.txt')
    
    triples_all = triples_tr + triples_va + triples_te
    
    return triples_all, triples_tr, triples_va, triples_te


def build_vocab(triples):
    params = {}
    
    e_set = {s for (s, p, o) in triples} | {o for (s, p, o) in triples}
    p_set = {p for (s, p, o) in triples}

    params['e_vocab_size'] = len(e_set)
    params['p_vocab_size'] = len(p_set)

    e2idx = {e: idx for idx, e in enumerate(sorted(e_set))}
    p2idx = {p: idx for idx, p in enumerate(sorted(p_set))}
    
    return e2idx, p2idx, params


def build_multi_label(triples_tr):
    s2p2o = {}
    for (_s, _p, _o) in triples_tr:
        s, p, o = e2idx[_s], p2idx[_p], e2idx[_o] 
        if s not in s2p2o:
            s2p2o[s] = {p: [o]}
        if p not in s2p2o[s]:
            s2p2o[s][p] = [o]
        if o not in s2p2o[s][p]:
            s2p2o[s][p].append(o)
    return s2p2o


def next_train_batch(triples_tr, e2idx, p2idx, s2p2o):
    for i in range(0, len(triples_tr), Config.batch_size):
        _triples_tr = triples_tr[i : i+Config.batch_size]
        x_s = np.asarray([e2idx[s] for (s, p, o) in _triples_tr], dtype=np.int32)
        x_p = np.asarray([p2idx[p] for (s, p, o) in _triples_tr], dtype=np.int32)
        y = []
        for (_s, _p, _o) in _triples_tr:
            s, p, o = e2idx[_s], p2idx[_p], e2idx[_o] 
            temp = np.zeros([len(e2idx)])
            temp[s2p2o[s][p]] = 1.
            y.append(temp)
        y = np.asarray(y)
        yield x_s, x_p, y


def train_input_fn(triples_tr, e2idx, p2idx, s2p2o):
    dataset = tf.data.Dataset.from_generator(
        lambda: next_train_batch(sklearn.utils.shuffle(triples_tr),
                                 e2idx,
                                 p2idx,
                                 s2p2o),
        (tf.int32, tf.int32, tf.float32),
        (tf.TensorShape([None]),
         tf.TensorShape([None]),
         tf.TensorShape([None, len(e2idx)])))
    iterator = dataset.make_one_shot_iterator()
    x_s, x_p, y = iterator.get_next()
    return {'s': x_s, 'p': x_p}, y

In [4]:
def o_next_batch(eval_triples,
                 entity_to_idx, 
                 predicate_to_idx):
    for _i, (s, p, o) in enumerate(eval_triples):
        s_idx, p_idx = entity_to_idx[s], predicate_to_idx[p]
        yield np.atleast_1d(s_idx), np.atleast_1d(p_idx)


def o_input_fn(eval_triples,
               entity_to_idx, 
               predicate_to_idx):
    dataset = tf.data.Dataset.from_generator(
        lambda: o_next_batch(eval_triples,
                             entity_to_idx, 
                             predicate_to_idx),
        (tf.int32, tf.int32),
        (tf.TensorShape([None,]),
         tf.TensorShape([None,])))
    iterator = dataset.make_one_shot_iterator()
    s, p = iterator.get_next()
    return {'s': s, 'p': p}


def evaluate_rank(model,
                  valid_triples,
                  test_triples,
                  all_triples,
                  entity_to_idx,
                  predicate_to_idx):

    for eval_name, eval_triples in [('test', test_triples)]:
        _scores_o = list(model.predict(
            lambda: o_input_fn(eval_triples,
                               entity_to_idx, 
                               predicate_to_idx)))

        ScoresO = np.reshape(_scores_o, [len(eval_triples), len(entity_to_idx)])
        ranks_o, filtered_ranks_o = [], []
    
        for _i, ((s, p, o), scores_o) in enumerate(zip(eval_triples, ScoresO)):
            s_idx, p_idx, o_idx = entity_to_idx[s], predicate_to_idx[p], entity_to_idx[o]

            ranks_o += [1 + np.sum(scores_o > scores_o[o_idx])]
            
            filtered_scores_o = scores_o.copy()
            rm_idx_o = [entity_to_idx[fo] for (fs, fp, fo) in all_triples if fs == s and fp == p and fo != o]
            filtered_scores_o[rm_idx_o] = - np.inf
            
            filtered_ranks_o += [1 + np.sum(filtered_scores_o > filtered_scores_o[o_idx])]

        for setting_name, setting_ranks in [('Raw', ranks_o), ('Filtered', filtered_ranks_o)]:
            mean_rank = np.mean(setting_ranks)
            print('[{}] {} Mean Rank: {}'.format(eval_name, setting_name, mean_rank))
            for k in [1, 3, 5, 10]:
                hits_at_k = np.mean(np.asarray(setting_ranks) <= k) * 100
                print('[{}] {} Hits@{}: {}'.format(eval_name, setting_name, k, hits_at_k))

In [ ]:
def alt_concat(x1_x2):
    return tf.concat(x1_x2, 1)


def forward(features, mode, params):
    batch_sz = tf.shape(features['s'])[0]
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    
    e_embed = tf.get_variable('e_embed',
                              [params['e_vocab_size'], Config.embed_dim],
                              initializer=tf.contrib.layers.xavier_initializer())
    p_embed = tf.get_variable('p_embed',
                              [params['p_vocab_size'], Config.embed_dim],
                              initializer=tf.contrib.layers.xavier_initializer())
    
    s = tf.nn.embedding_lookup(e_embed, features['s'])
    p = tf.nn.embedding_lookup(p_embed, features['p'])
    
    x = tf.map_fn(alt_concat, [tf.reshape(s, [10, batch_sz, 20, 1]),
                               tf.reshape(p, [10, batch_sz, 20, 1]),], dtype=tf.float32)
    x = tf.reshape(x, [batch_sz, 20, 20, 1])
    
    x = tf.layers.dropout(x, 0.2, training=is_training)
    x = tf.layers.conv2d(x, 32, (3,3), activation=tf.nn.elu)
    
    x = tf.layers.flatten(x)
    x = tf.layers.dropout(x, 0.5, training=is_training)
    x = tf.layers.dense(x, Config.embed_dim, tf.nn.elu)
    
    logits = tf.matmul(x, e_embed, transpose_b=True)
    return logits
    
    
def model_fn(features, labels, mode, params):
    logits = forward(features, mode, params)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        tf.logging.info('\n'+pprint.pformat(tf.trainable_variables()))
        
        global_step = tf.train.get_global_step()
        
        loss_op = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,
                                                                        labels=labels))
        
        lr_op = tf.train.exponential_decay(3e-3, global_step, 1000, 0.97)
        
        train_op = tf.train.AdamOptimizer(lr_op).minimize(loss_op,
                                                          global_step = global_step)
        
        lth = tf.train.LoggingTensorHook({'lr': lr_op,}, every_n_iter=100)
        
        return tf.estimator.EstimatorSpec(mode = mode,
                                          loss = loss_op,
                                          train_op = train_op,
                                          training_hooks = [lth])
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions = tf.sigmoid(logits))

In [ ]:
triples_all, triples_tr, triples_va, triples_te = load_triple()
e2idx, p2idx, params = build_vocab(triples_all)
s2p2o = build_multi_label(triples_tr)

model = tf.estimator.Estimator(model_fn,
                               params = params)

for n_epoch in range(Config.n_epochs):
    model.train(lambda: train_input_fn(triples_tr, e2idx, p2idx, s2p2o))
    evaluate_rank(model,
                  triples_va,
                  triples_te,
                  triples_all,
                  e2idx,
                  p2idx,)

Files Already Downloaded
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpok26vxxu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10e0db048>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>,
 <tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d/bi

INFO:tensorflow:Saving checkpoints for 2213 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpok26vxxu/model.ckpt.
INFO:tensorflow:loss = 1978.3218, step = 2213
INFO:tensorflow:lr = 0.0028045317
INFO:tensorflow:global_step/sec: 2.53251
INFO:tensorflow:loss = 2166.1614, step = 2313 (39.488 sec)
INFO:tensorflow:lr = 0.0027960024 (39.488 sec)
INFO:tensorflow:global_step/sec: 2.34359
INFO:tensorflow:loss = 1804.8466, step = 2413 (42.670 sec)
INFO:tensorflow:lr = 0.002787499 (42.671 sec)
INFO:tensorflow:global_step/sec: 2.33738
INFO:tensorflow:loss = 2142.2896, step = 2513 (42.782 sec)
INFO:tensorflow:lr = 0.0027790214 (42.782 sec)
INFO:tensorflow:global_step/sec: 2.20849
INFO:tensorflow:loss = 1805.1155, step = 2613 (45.280 sec)
INFO:tensorflow:lr = 0.0027705696 (45.279 sec)
INFO:tensorflow:global_step/sec: 2.34984
INFO:tensorflow:loss = 1743.753, step = 2713 (42.556 sec)
INFO:tensorflow:lr = 0.0027621435 (42.556 sec)
INFO:tensorflow:global_step/sec: 2.23599
INFO:tensorflow:loss = 1

INFO:tensorflow:lr = 0.0025509035 (41.301 sec)
INFO:tensorflow:global_step/sec: 2.42288
INFO:tensorflow:loss = 729.6574, step = 5425 (41.273 sec)
INFO:tensorflow:lr = 0.0025431453 (41.273 sec)
INFO:tensorflow:global_step/sec: 2.35876
INFO:tensorflow:loss = 694.5234, step = 5525 (42.395 sec)
INFO:tensorflow:lr = 0.002535411 (42.395 sec)
INFO:tensorflow:Saving checkpoints for 5530 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpok26vxxu/model.ckpt.
INFO:tensorflow:Loss for final step: 3.0755696.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpok26vxxu/model.ckpt-5530
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[test] Raw Mean Rank: 1813.6586
[test] Raw Hits@1: 17.32
[test] Raw Hits@3: 25.019999999999996
[test] Raw Hits@5: 28.360000000000003
[test] Raw Hits@10: 33.54
[test] Filtered Mean 

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpok26vxxu/model.ckpt-7742
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 7743 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpok26vxxu/model.ckpt.
INFO:tensorflow:loss = 276.05768, step = 7743
INFO:tensorflow:lr = 0.0023697806
INFO:tensorflow:global_step/sec: 2.37183
INFO:tensorflow:loss = 319.7936, step = 7843 (42.163 sec)
INFO:tensorflow:lr = 0.0023625733 (42.163 sec)
INFO:tensorflow:global_step/sec: 2.54924
INFO:tensorflow:loss = 299.51794, step = 7943 (39.227 sec)
INFO:tensorflow:lr = 0.0023553881 (39.227 sec)
INFO:tensorflow:global_step/sec: 2.43064
INFO:tensorflow:loss = 244.65103, step = 8043 (41.141 sec)
INFO:tensorflow:lr = 0.0023482246 (41.141 sec)
INFO:tensorflow:global_step/sec: 2.4

INFO:tensorflow:loss = 244.39986, step = 10655 (42.356 sec)
INFO:tensorflow:lr = 0.0021686398 (42.356 sec)
INFO:tensorflow:global_step/sec: 2.43006
INFO:tensorflow:loss = 179.20915, step = 10755 (41.151 sec)
INFO:tensorflow:lr = 0.0021620444 (41.151 sec)
INFO:tensorflow:global_step/sec: 2.36915
INFO:tensorflow:loss = 187.95294, step = 10855 (42.209 sec)
INFO:tensorflow:lr = 0.002155469 (42.209 sec)
INFO:tensorflow:global_step/sec: 2.37859
INFO:tensorflow:loss = 313.72797, step = 10955 (42.042 sec)
INFO:tensorflow:lr = 0.0021489137 (42.041 sec)
INFO:tensorflow:global_step/sec: 2.4195
INFO:tensorflow:loss = 247.52658, step = 11055 (41.331 sec)
INFO:tensorflow:lr = 0.0021423781 (41.331 sec)
INFO:tensorflow:Saving checkpoints for 11060 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpok26vxxu/model.ckpt.
INFO:tensorflow:Loss for final step: 4.3660293.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restor

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>,
 <tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(10368, 200) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(200,) dtype=float32_ref>]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpok26vxxu/model.ckpt-13272
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 13273 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpok26vxxu/model.ckpt.
INFO:tensorflow:loss = 81.56392, step = 13273
INFO:tensorflow:lr = 0.0020024232
INFO:tensorflow:global_step/se

INFO:tensorflow:loss = 76.50444, step = 15885 (43.062 sec)
INFO:tensorflow:lr = 0.0018492843 (43.062 sec)
INFO:tensorflow:global_step/sec: 2.35069
INFO:tensorflow:loss = 65.08342, step = 15985 (42.541 sec)
INFO:tensorflow:lr = 0.00184366 (42.541 sec)
INFO:tensorflow:global_step/sec: 2.36505
INFO:tensorflow:loss = 119.72062, step = 16085 (42.282 sec)
INFO:tensorflow:lr = 0.001838053 (42.282 sec)
INFO:tensorflow:global_step/sec: 2.25003
INFO:tensorflow:loss = 70.306435, step = 16185 (44.444 sec)
INFO:tensorflow:lr = 0.0018324631 (44.444 sec)
INFO:tensorflow:global_step/sec: 2.43341
INFO:tensorflow:loss = 119.60995, step = 16285 (41.095 sec)
INFO:tensorflow:lr = 0.0018268899 (41.095 sec)
INFO:tensorflow:global_step/sec: 2.28903
INFO:tensorflow:loss = 91.90924, step = 16385 (43.687 sec)
INFO:tensorflow:lr = 0.0018213339 (43.687 sec)
INFO:tensorflow:global_step/sec: 2.2465
INFO:tensorflow:loss = 99.91316, step = 16485 (44.513 sec)
INFO:tensorflow:lr = 0.0018157945 (44.513 sec)
INFO:tensorfl

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[test] Raw Mean Rank: 616.3296
[test] Raw Hits@1: 26.700000000000003
[test] Raw Hits@3: 38.879999999999995
[test] Raw Hits@5: 44.32
[test] Raw Hits@10: 52.019999999999996
[test] Filtered Mean Rank: 600.279
[test] Filtered Hits@1: 32.48
[test] Filtered Hits@3: 43.919999999999995
[test] Filtered Hits@5: 48.84
[test] Filtered Hits@10: 55.74
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>,
 <tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(10368, 200) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(200,) dtype=float32_ref>]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorfl